# Large Language Model for Smart Ocean Mobility Convergence<br>Assignment #3: Fine-Tuning GPT-2 for Text Classification

Deep learning lab, Naval Architecture & Ocean Engineering, Seoul National University.

## Problem Statement

<img src="./imgs/FineTuning.png" alt="finetuning" width="700"/><br>
In this assignment, you are required to fine-tune a pre-trained GPT-2 small model for a text classification task using two distinct fine-tuning methods.<br> This involves the following objectives:
- Loading a pre-trained GPT-2 small model and preparing the dataset for text classification.
- Implementing Feature Extraction Fine-Tuning by freezing most of the model's layers and training only the final layer.
- Implementing LoRA (Low-Rank Adaptation) Fine-Tuning, which involves applying low-rank updates to specific model parameters.
- Evaluating and comparing the performance of both fine-tuning approaches using metrics like accuracy and F1-score.
- Analyzing the results and discussing the trade-offs between the two methods.<br><br>


**Important**
- <font color=red>**DO NOT clear the final outputs**</font><br><br>

### Check virtual env and import packages

In [ ]:
!pip install fsspec[http]==2024.9.0

In [ ]:
!pip install transformers datasets accelerate peft

In [ ]:
import os
#assert os.environ["CONDA_DEFAULT_ENV"] == "2024_LLM", "current environment is not 2024_LLM"   # Feel free to modify it.

# Feel free to modify it.
%env CUDA_VISIBLE_DEVICES = 0

# Import required libraries
try:
    import torch
    print(f"Successfully imported torch. Version: {torch.__version__}")
except ImportError:
    print("Error importing torch.")

try:
    from datasets import load_dataset, load_metric
    print(f"Successfully imported load_dataset and load_metric from datasets.")
except ImportError:
    print("Error importing load_dataset and load_metric from datasets.")

try:
    from peft import LoraConfig, get_peft_model, TaskType
    print(f"Successfully imported LoraConfig, get_peft_model and TaskType from peft.")
except ImportError:
    print("Error importing LoraConfig, get_peft_model and TaskType from peft.")

try:
    from transformers import GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
    print(f"Successfully imported GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, Trainer, TrainingArguments and DataCollatorWithPadding from transformers.")
except ImportError:
    print("Error importing GPT2Tokenizer from transformers.")

env: CUDA_VISIBLE_DEVICES=0
Successfully imported torch. Version: 2.5.1+cu121
Error importing load_dataset and load_metric from datasets.
Successfully imported LoraConfig, get_peft_model and TaskType from peft.
Successfully imported GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, Trainer, TrainingArguments and DataCollatorWithPadding from transformers.


## Dataset Preparation and Preprocessing Overview

- Loads the IMDb dataset and initializes GPT-2 tokenizer with EOS token for padding.
- Applies tokenization with truncation and fixed-length padding (max length 512).
- Splits the tokenized dataset into training and testing sets.
- Uses a data collator for dynamic padding within batches.
- Loads accuracy and F1-score metrics for model evaluation.

In [ ]:
# Load the dataset
from datasets import load_dataset
from sklearn.metrics import f1_score, accuracy_score

dataset = load_dataset('imdb')

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Data preprocessing function
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
# Split the data
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

# Set up data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load metrics for accuracy and F1 score
accuracy_metric = accuracy_score
f1_metric = f1_score

In [ ]:
split_data = tokenized_datasets["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = split_data["train"]
validation_dataset = split_data["test"]

## Evaluation Metrics Function

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': acc, 'f1': f1}

## 1. Feature Extraction method

Objective: Fine-tune only the last layer of the GPT-2 model (Feature Extraction).<br><br>
Procedure:
- Load GPT2ForSequenceClassification with num_labels=2.
- Freeze all model parameters except for the final classification layer (score layer).
- Set training arguments (e.g., learning rate, batch size, number of epochs).
- Initialize Trainer with the training and evaluation datasets.
- Train the model using trainer.train().

In [ ]:
import os

# Disable W&B logging
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
pip uninstall wandb -y

Found existing installation: wandb 0.18.7
Uninstalling wandb-0.18.7:
  Successfully uninstalled wandb-0.18.7


In [ ]:
# Method 1: Feature Extraction
########## TODO: Set up the model and training arguments for training only the last layer ##########

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id

device = "cuda" if torch.cuda.is_available() else "cpu"

#Freeze all the layers except the last one, the base model contains all layers except the head layer
for param in model.base_model.parameters():
    param.requires_grad = False

lr = 1e-4
batch_size = 64
num_epochs = 5

training_args_last_layer = TrainingArguments(
    output_dir='./results_last_layer',
    num_train_epochs = num_epochs,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    warmup_steps = 500,
    weight_decay = 0.01,
    learning_rate = lr,
    logging_dir = './logs_last_layer',
    evaluation_strategy="epoch",
    logging_steps = 100
)

trainer_last_layer = Trainer(
    model = model,
    args = training_args_last_layer,
    train_dataset = train_dataset,
    eval_dataset = validation_dataset,  #the validation dataset is the same as the test dataset
    compute_metrics = compute_metrics
)

trainer_last_layer.train()

####################################################################################################

# Evaluate the model
results_last_layer = trainer_last_layer.evaluate(test_dataset)
trainer_last_layer.save_model('./last_layer_model')


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,1.087400
200,1.037200
300,0.922800
400,0.814300
500,0.677100
600,0.589700
700,0.525600
800,0.507700
900,0.489600
1000,0.473200


In [ ]:
print(results_last_layer)

{'eval_loss': 0.4410405457019806, 'eval_accuracy': 0.81432, 'eval_f1': 0.8141455180281217, 'eval_runtime': 805.3837, 'eval_samples_per_second': 31.041, 'eval_steps_per_second': 0.485, 'epoch': 5.0}


## 2. LoRA Fine-Tuning method

Objective: Apply LoRA (Low-Rank Adaptation) to fine-tune the GPT-2 model efficiently by updating only low-rank parameters.<br><br>
Procedure:
- Load GPT2ForSequenceClassification with num_labels=2.
- Configure LoRA settings using LoraConfig.
- Apply LoRA configuration to the model using get_peft_model().
- Define training arguments (e.g., learning rate, batch size, number of epochs).
- Initialize the Trainer with the training and evaluation datasets.
- Fine-tune the model using trainer.train().

In [1]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType
import torch
import os
from datasets import load_dataset
from sklearn.metrics import f1_score, accuracy_score

!pip install transformers datasets accelerate peft
!pip install fsspec[http]==2024.9.0


# Feel free to modify it.
%env CUDA_VISIBLE_DEVICES = 0

# Import required libraries
try:
    import torch
    print(f"Successfully imported torch. Version: {torch.__version__}")
except ImportError:
    print("Error importing torch.")

try:
    from datasets import load_dataset, load_metric
    print(f"Successfully imported load_dataset and load_metric from datasets.")
except ImportError:
    print("Error importing load_dataset and load_metric from datasets.")

try:
    from peft import LoraConfig, get_peft_model, TaskType
    print(f"Successfully imported LoraConfig, get_peft_model and TaskType from peft.")
except ImportError:
    print("Error importing LoraConfig, get_peft_model and TaskType from peft.")

try:
    from transformers import GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
    print(f"Successfully imported GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification, Trainer, TrainingArguments and DataCollatorWithPadding from transformers.")
except ImportError:
    print("Error importing GPT2Tokenizer from transformers.")

!pip uninstall wandb -y
os.environ["WANDB_DISABLED"] = "true"

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': acc, 'f1': f1}


dataset = load_dataset('imdb')


# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)


device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.is_available())

lora_config = LoraConfig(
    task_type="SEQ_CLASSIFICATION",
    target_modules=["c_fc", "c_proj"],  
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# Apply LoRA configuration to the model
model = get_peft_model(model, lora_config)

for name, param in model.named_parameters():
    if not any(target in name for target in ["c_fc", "c_proj"]):  # Leave these layers trainable
        param.requires_grad = False

# Set pad token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


# Preprocess datasets
def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples['text'],
        padding=False,  #The data collator adds the padding
        truncation=True,
        max_length=512,
    )
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']


# Initialize data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


lr = 1e-4
batch_size = 16
num_epochs = 5

# Set training arguments
training_args_lora = TrainingArguments(
    output_dir='/home/s0/ml4sys02/assignment3/results_lora',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=lr,
    logging_dir='/home/s0/ml4sys02/assignment3/logs_lora',
    evaluation_strategy="epoch",
    logging_steps=300,
    gradient_accumulation_steps=8,
    fp16=True,
    report_to="none",
)

# Initialize Trainer
trainer_lora = Trainer(
    model=model,
    args=training_args_lora,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,

)
# Start training
trainer_lora.train()

# Evaluate and save the model
results_lora = trainer_lora.evaluate()
trainer_lora.save_model('/home/s0/ml4sys02/assignment3/lora_model')

print(results_lora)




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
env: CUDA_VISIBLE_DEVICES=0
Successfully imported torch. Version: 2.5.1+cu121
Error importing load_dataset and load_metric from datasets.
Successfully imported Lo

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,No log
1,0.694900,No log
2,0.694900,No log
3,0.264200,No log
4,0.213500,No log


{'eval_runtime': 371.2704, 'eval_samples_per_second': 67.336, 'eval_steps_per_second': 4.21, 'epoch': 4.990403071017274}


In [21]:
import numpy as np

trainer_lora.save_model('/lora_model')
predictions_output = trainer_lora.predict(test_dataset)
logits = predictions_output.predictions[1]
print(f"Logits shape: {logits.shape}")
predicted_classes = np.argmax(logits, axis=1)


true_labels = np.array(test_dataset["label"])


Logits shape: (25000, 2)


In [22]:
from sklearn.metrics import accuracy_score, f1_score

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_classes)

# Calculate F1-score
f1 = f1_score(true_labels, predicted_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1-Score: {f1}")


Accuracy: 0.92852
F1-Score: 0.9285162529935429


In [ ]:
# Compare the results
#Unfortunatly this didnt work for me, I am not sure why.
print("Results of training only the last layer:", results_last_layer)
print("Results of LoRA fine-tuning:", results_lora)